# Regresión Lineal Múltiple: Calidad del Vino Tinto

Este proyecto analiza la relación entre diversas variables del vino tinto y su calidad, utilizando un modelo de **regresión lineal múltiple**.

Los datos provienen del **UCI Machine Learning Repository** y contienen 1,599 observaciones con 11 características y una variable de salida: la calidad del vino.

---

### 1. Importación de datos y análisis inicial

En este paso, cargamos el conjunto de datos `A1.4 Vino Tinto.csv`, revisamos su estructura y mostramos algunas filas iniciales para comprender mejor su contenido.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

data = pd.read_csv("A1.4 Vino Tinto.csv")
print(f"Dimensiones de los datos: {data.shape}")
display(data.head(5))

Dimensiones de los datos: (1599, 12)


,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


---

### 2. División de datos en entrenamiento y prueba

Para evaluar el modelo de manera confiable, dividimos los datos en un conjunto de entrenamiento (80%) y otro de prueba (20%). Esta separación es aleatoria para evitar sesgos en el modelo.

In [2]:
X = data.drop(columns=['calidad'])
y = data['calidad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=105)
print(f"Dimensiones del entrenamiento: {X_train.shape}")
print(f"Dimensiones de prueba: {X_test.shape}")


Dimensiones del entrenamiento: (1279, 11)
Dimensiones de prueba: (320, 11)


---

### 3. Selección de características hacia adelante

Se emplea el método de **selección hacia adelante** para identificar las características más relevantes que explican la calidad del vino, utilizando la métrica de R² como criterio de selección.

In [3]:
model = LinearRegression()
sfs_delante = SFS(model, k_features=(2, 8), forward=True, floating=False, scoring='r2', cv=10)
sfs_delante.fit(X_train, y_train)
caract_delante = list(sfs_delante.k_feature_names_)
print("Características (hacia adelante):", caract_delante)

Características (hacia adelante): ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


---

### 4. Entrenamiento del modelo con selección hacia adelante

Una vez seleccionadas las mejores características, entrenamos un modelo de regresión lineal utilizando únicamente estas variables.

In [4]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, features):
    model = LinearRegression()
    model.fit(X_train[features], y_train)
    y_pred = model.predict(X_test[features])
    r2 = r2_score(y_test, y_pred)
    print(f"R² del modelo con selección hacia adelante: {r2:.4f}")
    return model, r2, y_pred

model_delante, r2_delante, y_pred_delante = train_and_evaluate_model(X_train, X_test, y_train, y_test, caract_delante)



R² del modelo con selección hacia adelante: 0.3807


---

### 5. Selección de características hacia atrás

Ahora realizamos un proceso de **selección hacia atrás** para refinar aún más la selección de variables, eliminando aquellas que contribuyen menos a la predicción.

In [5]:
sfs_atras = SFS(model, k_features=(2, 5), forward=False, floating=False, scoring='r2', cv=10)
sfs_atras.fit(X_train[caract_delante], y_train)
caract_atras = list(sfs_atras.k_feature_names_)
print("Características (hacia atrás):", caract_atras)

Características (hacia atrás): ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


---

### 6. Entrenamiento del modelo con selección hacia atrás y visualización

Entrenamos nuevamente el modelo con las variables finales y comparamos su desempeño con el modelo de selección hacia adelante.

In [6]:
# Paso 6: Entrenar modelo con selección hacia atrás
model_atras, r2_atras, y_pred_backward = train_and_evaluate_model(X_train, X_test, y_train, y_test, caract_atras)



R² del modelo con selección hacia adelante: 0.3757


**Comparación de los modelos:**

In [7]:

# Comparación de los modelos
print("\nComparación de R² entre los modelos:")
print(f"Selección hacia adelante: R² = {r2_delante:.4f}")
print(f"Selección hacia atrás: R² = {r2_atras:.4f}")
if r2_delante > r2_atras:
    print("El modelo con selección hacia adelante tiene mejor desempeño.")
else:
    print("El modelo con selección hacia atrás tiene mejor desempeño.")



Comparación de R² entre los modelos:
Selección hacia adelante: R² = 0.3807
Selección hacia atrás: R² = 0.3757
El modelo con selección hacia adelante tiene mejor desempeño.


---

### Conclusión

Durante este análisis, se probaron varias veces el modelo con diferentes divisiones de datos (cambiando el random state), lo que permitió observar cómo varía la selección de características y el rendimiento del modelo. Se notó que la selección de características son importantes en la precisión de la regresión lineal múltiple, ya que tanto la selección hacia adelante como hacia atrás generaron resultados distintos en cada ejecución.

Un punto interesante es que la cantidad de variables elegidas afecta la estabilidad del modelo. En algunos casos, incluir más variables mejoró la predicción, mientras que en otros, reducirlas ayudó a evitar sobreajuste. Para mejorar la consistencia, sería útil ajustar mejor el número de características consideradas en cada método y comparar más a fondo los resultados.


**Referencias:**
- [UCI Machine Learning Repository: Wine Quality Dataset](https://archive.ics.uci.edu/dataset/186/wine+quality)
- [Decision Support Systems - Original Research Paper](https://doi.org/10.1016/j.dss.2009.05.016)